## Deep Neural Network for MNIST Classification

The dataset is called MNIST and refers to handwritten digit recognition. 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.

Our goal would be to build a neural network with 2 hidden layers.

In [1]:
## Importing the relevant libraries 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import tensorflow_datasets as tfds

In [2]:
# tfds.load actually loads a dataset
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# we will use this information a bit below and we will store it in mnist_info

# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary

mnist_dataset, mnist_info = tfds.load('mnist',with_info = True, as_supervised=True)

In [3]:
# once we have loaded the dataset, we can easily extract the training and testing dataset with the built references

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# we start by defining the number of validation samples as a % of the train samples
# Casting this number to an integer, as a float may cause an error along the way

num_validation_samples =  int(0.1*mnist_info.splits['train'].num_examples)

# Defining the number of validation samples using the same method

num_test_samples = int(mnist_info.splits['test'].num_examples)

In [4]:
# To have inputs between 0 and 1
# We define a function called: scale, that will take an MNIST image and its label

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [5]:
# Method .map() allows us to apply a custom transformation to a given dataset

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [6]:
# Shuffling the data
# Taking the validation data an train data from the shuffled data
buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [7]:
# Batch the test data
batch_size = 100

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [8]:
## Outline a model 

input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    # Each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # We must flatten the images
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=[28,28,1]),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size,activation='softmax')]) # output layer

In [9]:
## Compiling the model 

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS,
         validation_data=(validation_inputs,validation_targets),
         validation_steps=1, verbose=2)

Epoch 1/5
540/540 - 21s - loss: 0.4071 - accuracy: 0.8861 - val_loss: 0.2166 - val_accuracy: 0.9367
Epoch 2/5
540/540 - 14s - loss: 0.1783 - accuracy: 0.9478 - val_loss: 0.1593 - val_accuracy: 0.9502
Epoch 3/5
540/540 - 12s - loss: 0.1330 - accuracy: 0.9610 - val_loss: 0.1327 - val_accuracy: 0.9603
Epoch 4/5
540/540 - 13s - loss: 0.1103 - accuracy: 0.9675 - val_loss: 0.1008 - val_accuracy: 0.9703
Epoch 5/5
540/540 - 12s - loss: 0.0906 - accuracy: 0.9730 - val_loss: 0.1023 - val_accuracy: 0.9682


## Test the model 

In [11]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 4s 4s/step - loss: 0.1112 - accuracy: 0.9653

In [12]:
print("The loss is {0:.2f} and the accuracy is {1:.2f}".format(test_loss,test_accuracy*100))

The loss is 0.11 and the accuracy is 96.53
